# মিস্ট্রাল মডেল দিয়ে তৈরি করা

## পরিচিতি

এই পাঠে আলোচনা করা হবে:
- বিভিন্ন মিস্ট্রাল মডেল নিয়ে অনুসন্ধান
- প্রতিটি মডেলের ব্যবহার ক্ষেত্র ও পরিস্থিতি বোঝা
- কোড নমুনার মাধ্যমে প্রতিটি মডেলের বিশেষ বৈশিষ্ট্য দেখানো


## মিস্ট্রাল মডেলসমূহ

এই পাঠে, আমরা ৩টি ভিন্ন মিস্ট্রাল মডেল নিয়ে আলোচনা করব:
**Mistral Large**, **Mistral Small** এবং **Mistral Nemo**।

এই প্রতিটি মডেল Github Model মার্কেটপ্লেসে বিনামূল্যে পাওয়া যায়। এই নোটবুকে ব্যবহৃত কোডে এই মডেলগুলো ব্যবহার করা হবে। আরও বিস্তারিত জানতে, Github Models ব্যবহার করে [এআই মডেল দিয়ে প্রোটোটাইপ তৈরি](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst) করার নির্দেশিকা দেখুন।


## Mistral Large 2 (2407)
Mistral Large 2 বর্তমানে Mistral-এর প্রধান মডেল এবং এটি ব্যবসায়িক ব্যবহারের জন্য তৈরি।

এই মডেলটি মূল Mistral Large-এর তুলনায় উন্নত, কারণ এতে রয়েছে
-  বড় কনটেক্সট উইন্ডো - ১২৮k বনাম ৩২k
-  গণিত ও কোডিং টাস্কে আরও ভালো পারফরম্যান্স - গড় নির্ভুলতা ৭৬.৯% বনাম ৬০.৪%
-  বহুভাষিক পারফরম্যান্স বৃদ্ধি - অন্তর্ভুক্ত ভাষাসমূহ: ইংরেজি, ফরাসি, জার্মান, স্প্যানিশ, ইতালিয়ান, পর্তুগিজ, ডাচ, রুশ, চীনা, জাপানি, কোরিয়ান, আরবি এবং হিন্দি।

এই বৈশিষ্ট্যগুলোর কারণে, Mistral Large দক্ষ
- *Retrieval Augmented Generation (RAG)*-এ - বড় কনটেক্সট উইন্ডোর জন্য
- *Function Calling*-এ - এই মডেলে নেটিভ ফাংশন কলিং রয়েছে, যা বাইরের টুল ও API-এর সাথে সংযোগ করতে দেয়। এই কলগুলো একসাথে বা ধারাবাহিকভাবে একটির পর একটি করা যায়।
- *Code Generation*-এ - এই মডেলটি Python, Java, TypeScript এবং C++ কোড জেনারেশনে খুবই দক্ষ।


### মিস্ট্রাল লার্জ ২ ব্যবহার করে আরএজি উদাহরণ


এই উদাহরণে, আমরা Mistral Large 2 ব্যবহার করছি একটি টেক্সট ডকুমেন্টের উপর RAG প্যাটার্ন চালানোর জন্য। প্রশ্নটি কোরিয়ান ভাষায় লেখা এবং লেখকের কলেজে যাওয়ার আগের কার্যকলাপ সম্পর্কে জানতে চায়।

এখানে Cohere Embeddings Model ব্যবহার করা হয়েছে টেক্সট ডকুমেন্ট এবং প্রশ্নের এমবেডিং তৈরি করতে। এই নমুনার জন্য, faiss পাইথন প্যাকেজকে ভেক্টর স্টোর হিসেবে ব্যবহার করা হয়েছে।

Mistral মডেলে পাঠানো প্রম্পটে প্রশ্ন এবং প্রশ্নের সাথে মিল আছে এমন রিট্রিভ করা অংশগুলো অন্তর্ভুক্ত থাকে। এরপর মডেলটি স্বাভাবিক ভাষায় উত্তর প্রদান করে।


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## মিস্ট্রাল স্মল
মিস্ট্রাল স্মল হলো মিস্ট্রাল পরিবারের আরেকটি মডেল, যা প্রিমিয়ার/এন্টারপ্রাইজ ক্যাটাগরির অধীনে পড়ে। নাম থেকেই বোঝা যায়, এটি একটি স্মল ল্যাঙ্গুয়েজ মডেল (SLM)। মিস্ট্রাল স্মল ব্যবহারের সুবিধাগুলো হলো:
- খরচ সাশ্রয়ী, যেমন মিস্ট্রাল লার্জ ও NeMo-এর মতো মিস্ট্রাল LLM-এর তুলনায় ৮০% পর্যন্ত দাম কম
- কম লেটেন্সি - মিস্ট্রাল-এর LLM গুলোর তুলনায় দ্রুত রেসপন্স
- নমনীয় - বিভিন্ন পরিবেশে সহজেই ডিপ্লয় করা যায়, কম রিসোর্সের প্রয়োজনীয়তা ও কম সীমাবদ্ধতা নিয়ে

মিস্ট্রাল স্মল উপযুক্ত:
- টেক্সট ভিত্তিক কাজের জন্য, যেমন সারাংশ তৈরি, অনুভূতি বিশ্লেষণ ও অনুবাদ
- এমন অ্যাপ্লিকেশনের জন্য যেখানে খরচ সাশ্রয়ী হওয়ায় ঘন ঘন অনুরোধ করা হয়
- কম লেটেন্সি কোড সম্পর্কিত কাজের জন্য, যেমন কোড রিভিউ ও কোড সাজেশন


## মিস্ট্রাল স্মল এবং মিস্ট্রাল লার্জ তুলনা

মিস্ট্রাল স্মল এবং লার্জ-এর মধ্যে লেটেন্সির পার্থক্য দেখানোর জন্য নিচের সেলগুলো চালান।

আপনি ৩-৫ সেকেন্ডের মধ্যে রেসপন্স টাইমে পার্থক্য দেখতে পাবেন। একই প্রম্পটে রেসপন্সের দৈর্ঘ্য ও স্টাইলেও পার্থক্য লক্ষ্য করুন।


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## মিস্ট্রাল নেমো

এই পাঠে আলোচিত অন্য দুটি মডেলের তুলনায়, মিস্ট্রাল নেমো একমাত্র ফ্রি মডেল যেটির অ্যাপাচি২ লাইসেন্স রয়েছে।

এটি মিস্ট্রালের আগের ওপেন সোর্স LLM, মিস্ট্রাল 7B-এর একটি আপগ্রেড হিসেবে দেখা হয়।

নেমো মডেলের আরও কিছু বৈশিষ্ট্য হলো:

- *আরও দক্ষ টোকেনাইজেশন:* এই মডেলটি সাধারণত ব্যবহৃত tiktoken-এর পরিবর্তে Tekken টোকেনাইজার ব্যবহার করে। এর ফলে আরও বেশি ভাষা ও কোডে ভালো পারফরম্যান্স পাওয়া যায়।

- *ফাইনটিউনিং:* বেস মডেলটি ফাইনটিউনিংয়ের জন্য উপলব্ধ। এর ফলে এমন ব্যবহারিক ক্ষেত্রে আরও বেশি নমনীয়তা পাওয়া যায়, যেখানে ফাইনটিউনিং দরকার হতে পারে।

- *নেটিভ ফাংশন কলিং* - মিস্ট্রাল লার্জ-এর মতো, এই মডেলটিও ফাংশন কলিং-এ প্রশিক্ষিত। এটি এটিকে বিশেষ করে তোলে, কারণ এটি প্রথম ওপেন সোর্স মডেলগুলোর একটি, যেটি এই কাজটি করতে পারে।


## মিস্ট্রাল নেমো

এই পাঠে আলোচিত অন্য দুটি মডেলের তুলনায়, মিস্ট্রাল নেমো একমাত্র ফ্রি মডেল যেটির অ্যাপাচি২ লাইসেন্স রয়েছে।

এটি মিস্ট্রালের আগের ওপেন সোর্স LLM, মিস্ট্রাল 7B-এর একটি আপগ্রেড হিসেবে দেখা হয়।

নেমো মডেলের আরও কিছু বৈশিষ্ট্য হলো:

- *আরও দক্ষ টোকেনাইজেশন:* এই মডেলটি সাধারণত ব্যবহৃত tiktoken-এর পরিবর্তে Tekken টোকেনাইজার ব্যবহার করে। এর ফলে আরও বেশি ভাষা ও কোডে ভালো পারফরম্যান্স পাওয়া যায়।

- *ফাইনটিউনিং:* বেস মডেলটি ফাইনটিউনিংয়ের জন্য উপলব্ধ। এর ফলে এমন ব্যবহারিক ক্ষেত্রগুলোতে আরও বেশি নমনীয়তা পাওয়া যায়, যেখানে ফাইনটিউনিং দরকার হতে পারে।

- *নেটিভ ফাংশন কলিং* - মিস্ট্রাল লার্জ-এর মতো, এই মডেলটিও ফাংশন কলিং-এ প্রশিক্ষিত। এটি এটিকে অনন্য করে তোলে, কারণ এটি প্রথম ওপেন সোর্স মডেলগুলোর একটি, যেটি এই কাজটি করতে পারে।


### টোকেনাইজার তুলনা

এই উদাহরণে, আমরা দেখব কীভাবে Mistral NeMo টোকেনাইজেশন পরিচালনা করে Mistral Large-এর তুলনায়।

উভয় উদাহরণেই একই প্রম্পট নেওয়া হয়েছে, কিন্তু আপনি লক্ষ্য করবেন যে NeMo তুলনামূলকভাবে কম টোকেন ফেরত দেয় Mistral Large-এর চেয়ে।


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## শেখা এখানেই শেষ নয়, যাত্রা চালিয়ে যান

এই পাঠ শেষ করার পর, আমাদের [Generative AI Learning collection](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) দেখুন, যাতে আপনি আপনার Generative AI জ্ঞানে আরও দক্ষ হতে পারেন!



---

**দায়িত্ব অস্বীকার**:
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসম্ভব নির্ভুল অনুবাদের চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন, স্বয়ংক্রিয় অনুবাদে ভুল বা অসঙ্গতি থাকতে পারে। মূল ভাষায় রচিত নথিটিকেই চূড়ান্ত ও নির্ভরযোগ্য উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য পেশাদার মানব অনুবাদ গ্রহণ করার পরামর্শ দেওয়া হচ্ছে। এই অনুবাদের ব্যবহারে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা দায়ী থাকব না।
